In [1]:
from metaflow import Flow, namespace
namespace(None)
post_run =  next(Flow('HNSentimentAnalyzeComments').runs('analyze_this'))
sentiments = {}
comments_meta = {}
total_tokens = 0
for task in post_run['analyze_comments']:
    if 'post_sentiment' in task:
        for post_id, (score, num_tokens) in task['post_sentiment'].data.items():
            pid = int(post_id)
            total_tokens += num_tokens
            sentiments[pid] = score
            comments_meta[pid] = num_tokens
print(f"Sentiments found for {len(sentiments)} posts, {total_tokens} processed")

Sentiments found for 98505 posts, 212740652 processed


In [2]:
from collections import Counter
def all_topics(filterset=None):
    for post_id, (topic_lst, _) in topics.items():
        if filterset is None or post_id in filterset:
            for topic in topic_lst:
                yield topic.strip()

In [3]:
from metaflow import Flow, namespace
namespace(None)
post_run =  next(Flow('HNSentimentAnalyzePosts').runs('analyze_this'))
topics = {int(post_id): topics for post_id, topics in post_run['end'].task['post_tags'].data.items()}
post_run

Run('HNSentimentAnalyzePosts/argo-hnsentiment.user.tuulosgmail.com.hnsentimentanalyzeposts-rhdls')

In [4]:
from collections import defaultdict
topic_sentiment = defaultdict(list)
for post_id, (topic_lst, _) in topics.items():
    score = sentiments.get(post_id)
    if score is not None:
        for topic in topic_lst:
            topic = topic.strip()
            if topic and 'Please provide the article' not in topic:
                topic_sentiment[topic].append(score)

In [5]:
import duckdb
post_times = list(duckdb.query("select id, time, title, url from 'story.parquet' where to_timestamp(time) > timestamp '2020-01-01'").execute().fetchall())
post_meta = {id: (title, url, time) for id, time, title, url in post_times if id in sentiments}

In [6]:
from datetime import datetime
import pandas as pd
from collections import defaultdict

CUTOFF = datetime(2023, 6, 1).timestamp()

def timeseries():
    days = defaultdict(list)
    for post_id, tstamp, _, _ in post_times:
        if tstamp < CUTOFF:  
            if post_id in sentiments:
                day = datetime.fromtimestamp(tstamp).replace(hour=0, minute=0, second=0)
                days[day].append(sentiments[post_id])
    dfdata = []
    for day, series in days.items():
        series.sort()
        median = sum(series) / len(series)
        dfdata.append({'date': day, 'mood': median})  
    return pd.DataFrame(dfdata)

mood_df = timeseries()
mood_df.head()

,date,mood
0,2020-07-06,6.205882
1,2022-05-13,6.838235
2,2022-06-09,6.402597
3,2021-08-19,5.938776
4,2020-03-08,6.411765


In [7]:

import altair as alt

alt.renderers.enable('html')

from altair_theme import THEME
alt.themes.register('ob', lambda: THEME)
alt.themes.enable('ob')

base = alt.Chart(mood_df).properties(width=1000, height=700)
chart = base.mark_point().encode(x="date", y="mood")
chart + chart.transform_loess('date', 'mood').mark_line().encode(color=alt.value('#fc6603'))

alt.LayerChart(...)

In [8]:
from collections import Counter
#df = pd.DataFrame({'sentiment': list(sentiments.values())})

df = pd.DataFrame([{'score': s, 'count': c} for s, c in Counter(sentiments.values()).items()])

alt.Chart(df).mark_bar().encode(
    x=alt.X("score:O", title="Sentiment score"),
    y=alt.Y('count', title="Number of posts"),
).properties(width=600, height=300).configure(axisBottom=alt.AxisConfig(labelFontSize=16, titleFontSize=16, labelAngle=0),
                                              axisLeft=alt.AxisConfig(labelFontSize=14, titleFontSize=16))

alt.Chart(...)

In [9]:
import math
divisive = {}
for topic, scores in topic_sentiment.items():
    if len(scores) > 40:
        num_high = sum(1 for s in scores if s >= 7)
        num_low = sum(1 for s in scores if s <= 4)
        symmetric = 1.0 - abs(num_high - num_low) / (num_high + num_low)
        tail_heavy = (num_high + num_low) / len(scores)
        div = tail_heavy * symmetric
        divisive[topic] = div


In [11]:
from datetime import datetime
import json

def post_entry(arg, max_score=10):
    score, post_id = arg
    title, url, time = post_meta[post_id]
    return {
        'title': title,
        'url': f'https://news.ycombinator.com/item?id={post_id}',
        'time': datetime.fromtimestamp(time).strftime('%Y-%m-%d'),
        'score': score
    }

from collections import defaultdict
inv_index = defaultdict(list)
for post_id, (topic_lst, _) in topics.items():
    for topic in topic_lst:
        if post_id in sentiments and comments_meta.get(post_id, 0) > 200:
            inv_index[topic].append((sentiments[post_id], post_id))
            
topic_data = {}
for topic, scores in topic_sentiment.items():
    scores.sort()
    posts = sorted(inv_index[topic])
    if len(posts) > 5:
        n = len(posts) // 2
        topic_data[topic] = {
            'topic': topic,
            'num_posts': len(posts),
            'angry_posts': [post_entry((s, post_id)) for s, post_id in posts[:min(n, 5)] if s < 7],
            'happy_posts': list(map(post_entry, posts[max(-5, -n):])),
            'median_score': scores[len(scores) // 2],
            'divisiveness': divisive.get(topic, 0)
        }

with open('topic_data.json', 'w') as f:
    json.dump(topic_data, f)